In [1]:
import pandas as pd

In [2]:
# Importing tables 

productionData = pd.read_table(r"C:\Users\finng\OneDrive\Desktop\school\y32023-24\sTwo\Database\project\datasets\dataset\title.ratings.tsv")
productionData['averageRating'] = pd.to_numeric(productionData['averageRating'])
productionData['numVotes'] = pd.to_numeric(productionData['numVotes'])
Movie = pd.read_table(r"C:\Users\finng\OneDrive\Desktop\school\y32023-24\sTwo\Database\project\datasets\dataset\title.basics.tsv",low_memory=False)

In [3]:
# Adjusting data types

Movie['isAdult'] = Movie['isAdult'].astype(bool)
Movie['startYear'] = pd.to_numeric(Movie['startYear'].astype(str).str.replace('\.0', ''), errors='coerce').astype('Int64')
Movie['runtimeMinutes'] = pd.to_numeric(Movie['runtimeMinutes'], errors='coerce')

Movie['tconst'] = Movie['tconst'].astype(str)
Movie['tconst'] = Movie['tconst'].astype(str)

# Renaming columns

Movie.drop(columns='endYear', inplace=True)
Movie.drop(columns='originalTitle', inplace=True)
Movie.rename(columns={'startYear': 'year'}, inplace=True)
Movie.rename(columns={'tconst': 'movie_id'}, inplace=True)
Movie.rename(columns={'titleType': 'format'}, inplace=True)
Movie.rename(columns={'primaryTitle': 'movie_title'}, inplace=True)
Movie.rename(columns={'isAdult': 'is_adult'}, inplace=True)
Movie.rename(columns={'runtimeMinutes': 'run_time'}, inplace=True)
col = Movie.pop('movie_title')
Movie.insert(1, col.name, col)

productionData.rename(columns={'tconst': 'movie_id'}, inplace=True)
productionData.rename(columns={'averageRating': 'avg_rating'}, inplace=True)
productionData.rename(columns={'numVotes': 'num_votes'}, inplace=True)

In [4]:
# Creating Production_Details table

data = {'movie_id':productionData['movie_id'], 'details_id':'', 'avg_rating':productionData['avg_rating'], 'num_votes':productionData['num_votes'], 'year':Movie['year']}
Production_Details = pd.DataFrame(productionData)
years = {'movie_id':Movie['movie_id'], 'year':Movie['year']}
data = pd.DataFrame(years)
Production_Details = pd.merge(Production_Details, data, on="movie_id")
Movie.drop(columns='year', inplace=True)

In [5]:
# Creating Belongs relationship table

Belongs = Movie[['movie_id', 'genres']].copy()
Belongs = Belongs.assign(genres=Belongs['genres'].str.split(',')).explode('genres')
Movie.drop(columns='genres', inplace=True)

# Filling in missing values
Belongs['genres'] = Belongs['genres'].fillna('Unknown')

In [6]:
Belongs['genres'].replace(r'\N','Unknown',inplace=True)

In [7]:
# Filling in missing values

mean_year = int(Production_Details['year'].mean())
Production_Details['year'] = Production_Details['year'].fillna(mean_year)

mean_minutes = int(Movie['run_time'].mean())
Movie['run_time'] = Movie['run_time'].fillna(mean_minutes)

In [8]:
# Extracting unique genres for genre table

Belongs.rename(columns={'genres': 'genre_name'}, inplace=True)
data = {'genre_name': Belongs['genre_name'].unique()}
Genre=pd.DataFrame(data)
Genre['description']= ''

In [9]:
Production_Details.insert(0, 'details_id', range(1, 1 + len(Production_Details)))
Production_Details

,details_id,movie_id,avg_rating,num_votes,year
0,1,tt0000001,5.7,2017,1894
1,2,tt0000002,5.7,270,1892
2,3,tt0000003,6.5,1943,1892
3,4,tt0000004,5.4,178,1892
4,5,tt0000005,6.2,2721,1893
...,...,...,...,...,...
1396377,1396378,tt9916730,7.6,11,2017
1396378,1396379,tt9916766,7.1,23,2019
1396379,1396380,tt9916778,7.2,36,2019
1396380,1396381,tt9916840,8.8,6,2014


In [10]:
Movie

,movie_id,movie_title,format,is_adult,run_time
0,tt0000001,Carmencita,short,True,1.0
1,tt0000002,Le clown et ses chiens,short,True,5.0
2,tt0000003,Pauvre Pierrot,short,True,4.0
3,tt0000004,Un bon bock,short,True,12.0
4,tt0000005,Blacksmith Scene,short,True,1.0
...,...,...,...,...,...
10517942,tt9916848,Episode #3.17,tvEpisode,True,43.0
10517943,tt9916850,Episode #3.19,tvEpisode,True,43.0
10517944,tt9916852,Episode #3.20,tvEpisode,True,43.0
10517945,tt9916856,The Wind,short,True,27.0


In [11]:
Belongs

,movie_id,genre_name
0,tt0000001,Documentary
0,tt0000001,Short
1,tt0000002,Animation
1,tt0000002,Short
2,tt0000003,Animation
...,...,...
10517944,tt9916852,Family
10517945,tt9916856,Short
10517946,tt9916880,Adventure
10517946,tt9916880,Animation


In [12]:
Genre

,genre_name,description
0,Documentary,
1,Short,
2,Animation,
3,Comedy,
4,Romance,
5,Sport,
6,News,
7,Drama,
8,Fantasy,
9,Horror,


In [13]:
# Exporting json Files

import json

df = pd.DataFrame(Production_Details)
df.to_json('Production_Details.json',orient = 'split',compression = 'infer', index='false')

In [14]:
df = pd.DataFrame(Movie)
df.to_json('Movie.json',orient = 'split',compression = 'infer', index='false')

In [15]:
df = pd.DataFrame(Genre)
df.to_json('Genre.json',orient = 'split',compression = 'infer', index='false')

In [16]:
df = pd.DataFrame(Belongs)
df.to_json('Belongs.json',orient = 'split',compression = 'infer', index='false')

In [18]:
import csv
Genre.to_csv('Genre.csv')

In [19]:
Movie.to_csv('Movie.csv')

In [27]:
Movie.iloc[:1000000].to_csv('Movie01.csv')
Movie.iloc[1000001:2000000].to_csv('Movie02.csv')
Movie.iloc[2000001:3000000].to_csv('Movie03.csv')
Movie.iloc[3000001:4000000].to_csv('Movie04.csv')
Movie.iloc[4000001:5000000].to_csv('Movie05.csv')
Movie.iloc[5000001:6000000].to_csv('Movie06.csv')
Movie.iloc[6000001:7000000].to_csv('Movie07.csv')
Movie.iloc[7000001:8000000].to_csv('Movie08.csv')
Movie.iloc[8000001:9000000].to_csv('Movie09.csv')
Movie.iloc[9000001:10000000].to_csv('Movie10.csv')
Movie.iloc[10000001:].to_csv('Movie11.csv')

In [26]:
Belongs.iloc[:1227999].to_csv('Belongs01.csv')
Belongs.iloc[1228000:2000000].to_csv('Belongs02.csv')
Belongs.iloc[2000001:3000000].to_csv('Belongs03.csv')
Belongs.iloc[3000001:4000000].to_csv('Belongs04.csv')
Belongs.iloc[4000001:5000000].to_csv('Belongs05.csv')
Belongs.iloc[5000001:6000000].to_csv('Belongs06.csv')
Belongs.iloc[6000001:7000000].to_csv('Belongs07.csv')
Belongs.iloc[7000001:8000000].to_csv('Belongs08.csv')
Belongs.iloc[8000001:9000000].to_csv('Belongs09.csv')
Belongs.iloc[10000001:9000000].to_csv('Belongs10.csv')
Belongs.iloc[10000001:11000000].to_csv('Belongs11.csv')
Belongs.iloc[11000001:12000000].to_csv('Belongs12.csv')
Belongs.iloc[12000001:13000000].to_csv('Belongs13.csv')
Belongs.iloc[13000001:14000000].to_csv('Belongs14.csv')
Belongs.iloc[14000001:15000000].to_csv('Belongs15.csv')
Belongs.iloc[15000001:16000000].to_csv('Belongs16.csv')
Belongs.iloc[16000001:].to_csv('Belongs17.csv')

In [ ]:
Belongs.to_csv('Belongs.csv')

In [23]:
Production_Details.iloc[:500000].to_csv('Production_Details1.csv')
Production_Details.iloc[500001:].to_csv('Production_Details2.csv')

In [24]:
Production_Details.to_csv('Production_Details.csv')

In [25]:
Movie.describe()

,run_time
count,1.051795e+07
mean,4.319144e+01
std,4.073265e+01
min,0.000000e+00
25%,4.300000e+01
50%,4.300000e+01
75%,4.300000e+01
max,5.432100e+04
